In [1]:
# | default_exp classes.DomoAccessToken

In [2]:
# | exporti
import httpx
import datetime as dt
from nbdev.showdoc import patch_to

from dataclasses import dataclass, field, asdict

import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de
import domolibrary.utils.chunk_execution as ce

import domolibrary.routes.access_token as access_token_routes

In [3]:
# |hide
from nbdev.showdoc import show_doc
import os

# Access Token

In [4]:
# | export


@dataclass
class DomoAccessToken:
    id: int
    name: str
    owner: None
    expiration_date: dt.datetime
    token : str = field(repr = False)

    auth: dmda.DomoAuth = field(repr = False)

    def __eq__(self, other):
        if not isinstance(other, DomoAccessToken):
            return False

        return self.id == other.id

    @classmethod
    async def _from_json(cls, obj, auth):
        import domolibrary.classes.DomoUser as dmu

        owner = await dmu.DomoUser.get_by_id(user_id=obj["ownerId"], auth=auth)

        return cls(
            id=obj["id"],
            name=obj["name"],
            owner=owner,
            expiration_date=obj["expires"],
            auth=auth,
            token = obj.get('token')
        )

In [5]:
# | export
async def get_access_tokens(
    auth: dmda.DomoAuth = None,
    return_raw: bool = False,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
    session: httpx.AsyncClient = None,
    parent_class=None,
):
    res = await access_token_routes.get_access_tokens(
        auth=auth,
        session=session,
        debug_api=debug_api,
        parent_class=parent_class,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if return_raw:
        return res

    return await ce.gather_with_concurrency(
        *[DomoAccessToken._from_json(obj=obj, auth=auth) for obj in res.response], n=10
    )

In [6]:
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_tokens = await get_access_tokens(auth=token_auth)
domo_tokens[0:5]

warning this token has not been validated by who_am_i, run get_auth_token first


[DomoAccessToken(id=156180, name='Governance', owner=DomoUser(id='587894148', display_name='Bryan Van Kampen', email_address='bryan@bvankampen.net', role_id=1, department='BT Partners', title='Executive Analytics', avatar_key='09/6CF6C801EEF440968C88ECC9824D66', phone_number='6304647034', web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=1588827600000, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None, custom_attributes={}, role=DomoRole(id=1, name='Admin', description='Full access to everything', is_system_role=True, is_default_role=None, grant_ls=[], membership_ls=[])), expiration_date=datetime.datetime(2030, 5, 13, 14, 47, 33)),
 DomoAccessToken(id=159191, name='Java CLI', owner=DomoUser(id='1345408759', display_name='Alexis Lorenz (DataMaven)', email_address='DataMaven42@gmail.com', role_id=1, department='Owner', title='Major Domo Support Specialist', avatar_key='3ad4c694-c9e8-4b6b-8b80-366d347d

In [7]:
# | exporti
@patch_to(DomoAccessToken, cls_method=True)
async def generate(
    cls: DomoAccessToken,
    duration_in_days: int,
    token_name: str,
    auth: dmda.DomoAuth,
    owner: None,  # DomoUser
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    debug_num_stacks_to_drop: int = 2,
    return_raw: bool = False,
    parent_class: str = None,
):

    res = await access_token_routes.generate_access_token(
        user_id=owner.id,
        token_name=token_name,
        duration_in_days=duration_in_days,
        auth=auth,
        debug_api=debug_api,
        session=session,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class or cls.__name__,
    )

    if return_raw:
        return res

    return await cls._from_json(obj=res.response, auth=auth)

In [8]:
# | exporti


@patch_to(DomoAccessToken)
async def revoke(
    self,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    debug_num_stacks_to_drop: int = 2,
):
    return await access_token_routes.revoke_access_token(
        auth=self.auth,
        access_token_id=self.id,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=self.__class__.__name__,
        session=session,
    )

In [24]:
import domolibrary.classes.DomoUser as dmu

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_user = await dmu.DomoUsers.by_email(email_ls=["test4@domo.com"], auth=token_auth)

token = await DomoAccessToken.generate(
    token_name=f"DL test {dt.date.today()}",
    owner=domo_user,
    duration_in_days=30,
    auth=token_auth,
    debug_api=False,
)
token

warning this token has not been validated by who_am_i, run get_auth_token first


DomoAccessToken(id=186851, name='DL test 2024-03-20', owner=DomoUser(id='1216550715', display_name='8:26 - go to sleep', email_address='test4@domo.com', role_id=2, department='test', title=None, avatar_key=None, phone_number='152', web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None, custom_attributes={}, role=DomoRole(id=2, name='Privileged', description='Full access except for editing users and settings owners', is_system_role=True, is_default_role=None, grant_ls=[], membership_ls=[])), expiration_date=1713553489000)

In [22]:
# | exporti


@patch_to(DomoAccessToken)
async def regenerate(
    self,
    session: httpx.AsyncClient = None,
    duration_in_days: int = 90,
    debug_api: bool = False,
    return_raw: bool = False,
    debug_num_stacks_to_drop: int = 2,
):

    await self.revoke()

    return await self.generate(
        duration_in_days=duration_in_days,
        token_name=self.name,
        auth=self.auth,
        owner=self.owner,
        debug_api=debug_api,
        session=session,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        return_raw=return_raw,
        parent_class=self.__class__.__name__,
    )

In [25]:
#| eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await token.regenerate()

DomoAccessToken(id=186852, name='DL test 2024-03-20', owner=DomoUser(id='1216550715', display_name='8:26 - go to sleep', email_address='test4@domo.com', role_id=2, department='test', title=None, avatar_key=None, phone_number='152', web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None, custom_attributes={}, role=DomoRole(id=2, name='Privileged', description='Full access except for editing users and settings owners', is_system_role=True, is_default_role=None, grant_ls=[], membership_ls=[])), expiration_date=1718737497000)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export("./50_DomoAccessToken.ipynb")